### CPU-bound(cpu密集型)
指I/O在很短的时间就可以完成，CPU需要大量的计算和处理，特点是CPU占用率相当高
### I/O-bound(IO密集型)
指的是系统运作大部分的状况是CPU在等I/O（硬盘/内存/网络）的读/写操作，CPU占用率仍然较低。  
例如：文件处理程序，网络爬虫程序，读写数据库程序  
多线程、多进程、多协程的对比  
### 1.多进程 multiprocessing
优点：可以利用多核CPU并行运算  
缺点：占用资源最多、可启动数目比线程少  
适用于：CPU密集型计算  
### 2.多线程thread(threading)
**优点**：相比进程，更轻量级、占用资源少  
**缺点**：  
相比进程：多线程只能并发执行，不能利用多CPU（GIL）  
相比协程：启动数目有限制，占用内存资源，有线程切换开销  
适用于：I/O密集型计算、同时运行的任务数目要求不多  
### 3.多协程Coroutine(asyncio)
优点：内存开销最少，启动协程数量最多
缺点：支持的库有限制(aiohttp vs requests)、代码实现复杂
适用于：IO密集型计算，需要超多任务运行、但有现成库支持的场景
### Python 慢的两大原因
1.动态类型语言（要一直检查变量类型），边解释边执行  
2.GIL（Global Interpreter Lock）：无法利用多核CPU并发执行  
GIL:是计算机程序设计语言解释器用于同步线程的一种机制，它使得任何时刻仅有一个线程在执行。即便在多核处理器上，使用GIL的解释器也只允许同一时间执行一个线程。
由于GIL的存在，即时电脑有多核CPU，单个时刻也只能使用1个，相比于并发加速的C++/JAVA所以慢。  
GIL的好处：解决了多线程之间数据完整性和状态同步问题。简化了python对共享资源的管理    
**怎样规避GIL带来的限制？**  
1.多线程`threadin` 机制依然有用，用于I/O密集型计算  
因为在I/O期间，线程会释放GIL，实现CPU和IO的并行，因此多线程用于IO密集型计算依然可以大幅度提升速度。但在多线程用于CPU密集型计算时，只会拖慢速度，因为只有一个CPU在运行，同时进行多线程的切换，多线程的切换会带来开销  
2.使用`multiprocessing` 的多进程机制实现了并行计算，利用多核CPU优势，为了应对GIL问题，`Python`提供了`multiprocessing`模块  
相比于`C/C++/JAVA`，`python`确实慢，在一些特殊场景下，`Python`比`C++`慢100～200倍，由于速度慢的原因，很多公司的基础框架代码依然用C/C++开发。比如各大公司阿里/腾讯/快手的推荐引擎、搜索引擎、存储引擎等底层对性能要求高的模块。
### 多线程数据通信的`queue.Queue ` 
`queue.Queue`可以用于多线程之间、线程安全的数据通信，线程安全是指多个线程并发同时的访问数据不会产生冲突。  
```
import requests
from bs4 import BeautifulSoup
import threading
import time
import queue

urls = [f"https://www.cnblogs.com/#p{page}" for page in range(1, 50+1)]

def craw(url):
  	r = requests.get(url)
  	return r.text

def parse(html):
      soup = BeautifulSoup(html,'lxml')
      links = soup.find_all('a', class_='post-item-title')
      return [(link['href'], link.get_text()) for link in links]

def do_craw(url_queue:queue.Queue, html_queue:queue.Queue):
  	while True:
        url = url_queue.get()  #get url from url_queue
        html = craw(url)
        html_queue.put(html)   #put html in html_queue
        print(threading.current_thread().name, f'craw {url}', 'url_queue.size=', url_queue.qsize())

def do_parse(html_queue:queue.Queue, fout):
  	while True:
        html = html_queue.get()
        results = parse(html)
        for result in results:
          fout.write(str(result)+'\n')
        print(threading.current_thread().name, f'results.size',len(results),
              'html_queue.size=', html_queue.qsize())

def single_thread():
  	for url in urls:
    	craw(url)
    
def multi_thread():
  	threads = []
      for url in urls:
        threads.append(threading.Thread(target=craw, args=(url,)))
      for thread in threads:
        thread.start()
      for thread in threads:
        thread.join()

if __name__=="__main__":
      url_queue = queue.Queue()
      html_queue = queue.Queue()
      for url in urls:
        url.queue.put(url)

  #	启动3个生产者线程
  	for idx in range(3):
        t = threading.Thread(target=do_craw, args=(url_queue,html_queue),name=f'craw{idx}')
        t.start()

      fout = open('data.txt', 'w')
      #启动2个消费者线程
      for idx in range(2):
            t = threading.Thread(target=do_parse, args=(html_queue,fout),name=f'parse{idx}')
            t.start()

      start = time.time()
      single_thread()
      end = time.time()
      print('single thread cost:', end-start, 'second')

      start = time.time()
      multi_thread()
      end = time.time()
      print('multi thread cost:', end-start, 'second')
```
**线程安全** ：指某个函数、函数库在多线程环境中被调用时，能够正确处理多个线程之间的共享变量，使程序功能正常完成  
**线程不安全**：由于线程的执行随时会发生切换，就造成了不可预料的结果，出现线程不安全。  
```
import threading
import time

lock = threading.Lock()
class Account:
  	def __init__(self.balance):
    	self.balance = balance
    
def draw(account, amount):
  	with lock:
    	if account.balance >= amount:
      	time.sleep(0.1)
      	print(threading.current_thread().name, '取钱成功')
      	account.balance -= amount
      	print(threading.current_thread().name, '余额', account.balance)
    else:
      	print(threading.current_thread().name, '取钱失败，余额不足')
      
if __name__=='__main__':
  	account = Account(1000)
  	ta = threading.Thread(name='ta', target=draw, args=(account, 800))
  	tb = threading.Thread(name='tb', target=draw, args=(account, 800))
```
### 线程池
**1.提升性能**：因为减去了大量新建、终止线程的开销，重用了线程资源  
**2.使用场景**：适合处理突发性大量请求和需要大量线程完成任务、但实际任务处理时间较短  
**3.防御功能**：能有效避免系统因为创建线程过多，而导致系统负荷过大响应变慢等问题  
**4.代码优势**：使用线程池的语法比自己新建线程执行线程更加简洁  
```
import concurrent.futures
#craw
with concurrent.futures.TheadPoolExecutor() as pool:
  	htmls = pool.map(craw, urls)
  	htmls = list(zip(urls, htmls))
  	for url, html in htmls:
    	print(url, len(html))
print('craw over')

#parse
with concurrent.futures.TheadPoolExecutor() as pool:
  	futures = {}
  	for url, html in htmls:
    	futurn = pool.submit(parse, html)
    	futures[future] = url
    
  	#for future, url in futures.items():
  	#  print(url, future.result())
  	for future in concurrent.futures.as_completed(futures):
    	url = futures[future]
    	print(url, future.result())
```
**有了多线程threading,为什么还要用多进程multiprocessing**  
multiprocessing模块就是python为了解决GIL缺陷引入的一个模块，原理是用多进程在多CPU上并行执行。  
### 多进程
```
#判断一个数是否为素数
import math
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

PRIMES = [2389432] * 100
def is_prime(n):
  	if n < 2:
    	return False
  	if n == 2:
    	return True
  	if n % 2 == 0:
    	return False
  	sqrt_n = int(math.floor(math.sqrt(n)))
  	for i in range(3, sqrt_n + 1, 2):
    	if n % i == 0:
      	return False
  	return True

def single_thread():
  	for number in PRIMES:
    	is_prime(number)
    
def multi_thread():
  	with ThreadPoolExecutor() as pool:
    	pool.map(is_prime, PRIMES)
def multi_process():
  	with ProcessPoolExecutor() as pool:
    	pool.map(is_prime, PRIMES)

if __name__=='__main__':
  	pass
```
### python异步IO实现并发爬虫
```
import asyncio
import aiohttp

async def async_craw(url):
  	print('craw url:', url)
  	async with aiohttp.ClientSession() as session:
    	async with session.get(url) as resp:
      	result = await resp.text()
      	print(f'craw url:{url},{len(result)}')

loop = asyncio.get_event_loop()
urls = [f"https://www.cnblogs.com/#p{page}" for page in range(1, 50+1)]
tasks = [loop.creat_task(async_craw(url)) for url in urls]
loop.run_until_complete(asyncio.wait(tasks))
```